# Transformer 组件 

## FFN（前馈神经网络）


![FFN](./img/pwFFN.png)

Position-wise 实际是线性层本身的一个特性，在线性层中，每个输入向量（对应于序列中的一个位置，比如一个词向量）都会通过相同的权重矩阵进行线性变换，这意味着每个位置的处理是相互独立的，逐元素这一点可以看成 kernal_size=1 的卷积核扫过一遍序列。

FFN实现很简单，本质上是proj_up换加上激活函数，再加上一个proj_down


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        """
        FFN
        args:
            d_model: 输入和输出向量的维度
            d_ff： FFN隐藏层的维度
            dropout：随机屏蔽部分输出，防止过拟合（也是一种正则化手段）
        """
        super(PositionwiseFeedForward,self).__init__()
        self.proj_up = nn.Linear(d_model, d_ff)
        self.proj_down = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.proj_up(x).relu()
        x = self.dropout(x)
        x = self.proj_down(x)
        return x

In [8]:
batch_size = 64
seq_len = 256
d_model = 512
d_ff = 2048
x = torch.randn(batch_size, seq_len, d_model)

ffn = PositionwiseFeedForward(d_model, d_ff)
print("x shape:", x.shape, "\nffn(x) shape:", ffn(x).shape)

x shape: torch.Size([64, 256, 512]) 
ffn(x) shape: torch.Size([64, 256, 512])


## 残差连接

残差连接是一种跳跃连接，将输入直接加入到输出上（实际上，有了残差连接后参数的更新只需要去做f(x)-x的部分即可？）：
$$\text{Output} = \text{Sublayers}(x) + x$$
主要作用是**缓解梯度消失/爆炸**

In [4]:
class ResidualConnection(nn.Module):
    def __init__(self, dropout=0.1):
        """
        residual，用于在每个子层后添加残差连接和 Dropout。
        
        args:
            dropout: 防止过拟合。
        """
        super(ResidualConnection, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, sublayer):
        """
        args:
            x: 残差连接的输入张量，形状为 (batch_size, seq_len, d_model)。
            sublayer: 子层模块的函数，多头注意力或前馈网络。

        return:
            经过残差连接和 Dropout 处理后的张量，形状为 (batch_size, seq_len, d_model)。
        """
        # 将子层输出应用 dropout，然后与输入相加（参见论文 5.4 的表述或者本文「呈现」部分）
        return x + self.dropout(sublayer(x))

In [10]:
batch_size = 64
seq_len = 256
d_model = 512
d_ff = 2048
x = torch.randn(batch_size, seq_len, d_model)
ffn = PositionwiseFeedForward(d_model, d_ff)
residual = ResidualConnection()
print("x shape:", x.shape, "\nffn(x) and residual shape:", residual(x, ffn).shape)

x shape: torch.Size([64, 256, 512]) 
ffn(x) and residual shape: torch.Size([64, 256, 512])
